In [10]:
# BLOCK 1 #

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('test_stocks').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
!pip install yfinance

import pandas as pd
stock_timeline = pd.DataFrame.from_records(rows,columns=['Type', 'Action', 
                                                         'Action Date', 'Ticker','Quantity','Cost','Commission'])
stock_timeline = stock_timeline.iloc[1: , :]
stock_timeline['Action'] = stock_timeline['Action'].str.capitalize()
stock_timeline['Action Date'] = pd.to_datetime(stock_timeline['Action Date'].str.strip(),errors='coerce')
stock_timeline['Quantity'] = pd.to_numeric(stock_timeline['Quantity'])
stock_timeline['Additional Quantity'] = stock_timeline['Quantity']
stock_timeline['Cost'] = pd.to_numeric(stock_timeline['Cost'])
stock_timeline['Commission'] = pd.to_numeric(stock_timeline['Commission'])


stock_timeline.head()
print(stock_timeline.loc[1]['Action Date'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2002-06-08 00:00:00


In [ ]:
# BLOCK 2 #


#calculates gains from each sell, according to FIFO sell method
def total_cost(x, sell_index):

      new_returns = 0

      #loops continues until the quantity of stocks that need to be sold is zero
      while complete_stock_df.iat[sell_index, complete_stock_df.columns.get_loc('Unsold Quantity')] > 0:
        
        
        #identifies the row of the earliest purchased 
        first_unsold_index = complete_stock_df[complete_stock_df['Unsold Quantity'] > 0].index[0]
        if first_unsold_index >= sell_index:
          return

        #updates the unsold quantity field for the first unsold index, and computes the gain from the sell for the specific purchase date 
        previous_unsold_amount = complete_stock_df.iloc[first_unsold_index]['Unsold Quantity']
        quantity_difference = complete_stock_df.iloc[first_unsold_index]['Unsold Quantity'] - complete_stock_df.iat[sell_index, complete_stock_df.columns.get_loc('Unsold Quantity')]

        complete_stock_df.iat[first_unsold_index, complete_stock_df.columns.get_loc('Unsold Quantity')] = quantity_difference if quantity_difference > 0 else 0

        complete_stock_df.iat[sell_index, complete_stock_df.columns.get_loc('Unsold Quantity')] = quantity_difference * -1 if quantity_difference < 0 else 0
        total_sold = previous_unsold_amount - complete_stock_df.iloc[first_unsold_index]['Unsold Quantity']
        #new returns represents the cost of each share for the current buy multiplied by the quantity that is now being sold
        new_returns += total_sold * complete_stock_df.iloc[first_unsold_index]['Cost']

        
      complete_stock_df['Cumulative Cost'] = complete_stock_df['Transaction Val'].cumsum()
      complete_stock_df.iat[sell_index,complete_stock_df.columns.get_loc('Gains')] = x['Quantity'] * -1 * x['Cost'] - new_returns - x['Commission']
        

In [16]:
# BLOCK 3 #


#process each row, calculate the new cumulative cost. 
#process sell if row designated as such
def calculate_cumulative(row, row_index):

  if row['Action'] == 'Sell':
    total_cost(row, row_index)

  #previous_cum_quant is the cumulative quantity of stocks, including those sold, till date
  previous_cum_quant = complete_stock_df.iat[row_index-1,complete_stock_df.columns.get_loc('Cumulative Quant')]
  #previous_buys_quant is the cumulative quantity of stocks bought. This quantity does not account for those sold 
  previous_buys_quant = complete_stock_df.iat[row_index-1,complete_stock_df.columns.get_loc('Cumulative Buys')]

  #update both cumulative quantities to account for current quantity in the row and for a split
  complete_stock_df.iat[row_index,complete_stock_df.columns.get_loc('Cumulative Quant')] = previous_cum_quant * row['Split'] + row['Quantity']
  complete_stock_df.iat[row_index,complete_stock_df.columns.get_loc('Cumulative Buys')] = previous_buys_quant * row['Split'] + row['Additional Quantity']

  
  if row['Split'] > 1:
    #[1]
    #if split occurs, find all previous dividend actions and multiply their respective 
    #costs by the split multiple to account for Yahoo Finance database discrepancy 
    complete_stock_df.loc[(complete_stock_df['Action'] == 'Div') & (complete_stock_df['Action Date'] < row['Action Date']) , 'Cost'] = complete_stock_df[(complete_stock_df['Action'] == 'Div') & (complete_stock_df['Action Date'] < row['Action Date'])]['Cost'] * row['Split']

    #for all current holdings of the stock, multiply the unsold quantity by the split multiple, 
    #and divide the cost of those stocks by the split multiple
    first_unsold_df = complete_stock_df[complete_stock_df['Unsold Quantity'] > 0]
    if first_unsold_df.empty:
      return
    else:
      first_unsold_index = first_unsold_df.index[0]

    complete_stock_df.loc[(complete_stock_df['Unsold Quantity'] > 0) & (complete_stock_df['Action Date'] < row['Action Date']), 'Cost'] = complete_stock_df[(complete_stock_df['Unsold Quantity'] > 0) & (complete_stock_df['Action Date'] < row['Action Date'])]['Cost'] / row['Split']

    complete_stock_df.loc[(complete_stock_df['Unsold Quantity'] > 0) & (complete_stock_df['Action Date'] < row['Action Date']), 'Unsold Quantity'] = complete_stock_df[(complete_stock_df['Unsold Quantity'] > 0) & (complete_stock_df['Action Date'] < row['Action Date'])]['Unsold Quantity'] * row['Split']




In [11]:
# BLOCK 4 #

#stores split and dividend data in a separate dictionary for later reference

import yfinance as yf

unique_stocks = stock_timeline.Ticker.unique()
stock_to_split = {}
stock_to_dividends = {}
for stock in unique_stocks:
  yf_ticker = yf.Ticker(stock)
  stock_to_split[stock] = yf_ticker.splits
  stock_to_dividends[stock] = yf_ticker.dividends

  


- AWGI: No data found, symbol may be delisted
- AWGI: No data found, symbol may be delisted
- QLTI: No data found for this date range, symbol may be delisted
- QLTI: No data found for this date range, symbol may be delisted
- FLML: No data found for this date range, symbol may be delisted
- FLML: No data found for this date range, symbol may be delisted
- FDC: No data found, symbol may be delisted
- FDC: No data found, symbol may be delisted
- EPIQ: No data found for this date range, symbol may be delisted
- EPIQ: No data found for this date range, symbol may be delisted
- BBBB: No data found for this date range, symbol may be delisted
- BBBB: No data found for this date range, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- YHOO: No data found for this date range, symbol may be delisted
- YHOO: No data found for this date range, symbol may be delisted
- SCSS: No data found for th

In [12]:
# BLOCK 5 #

#prompts user to input which stocks they would like analyzed from their portfolio. 
#User can input 'whole!' to analyze the entire portfolio

chosen_stocks = [item.upper() for item in input("Enter list of stocks to analyze. Enter 'whole!' to analyze whole portfolio:  ").split(",")]
if len(chosen_stocks) == 1 and chosen_stocks[0] == 'WHOLE!':
  unique_stocks = stock_timeline.Ticker.unique()
  chosen_stocks = unique_stocks 

Enter list of stocks to analyze. Enter 'whole!' to analyze whole portfolio:  AAPL


In [17]:
# BLOCK 6 #


import yfinance as yf

from numpy.ma.core import cumprod
stock_transactions = pd.DataFrame()
stock_to_df = {}
overall_df = pd.DataFrame(columns=['Stock','Total Value', 'Outstanding Cost', 'Total Costs','Rate of Return %'])
#loop through each stock from stock timeline
count = 1
for stock in chosen_stocks:

  complete_stock_df = pd.DataFrame()
  stock_df = stock_timeline.loc[(stock_timeline['Ticker'] == stock) & (stock_timeline['Action'].isin(['Buy','Sell']))]
  stock_df.reset_index(inplace=True, drop = True)

  print('#: ', count, 'Ticker: ', stock)
  count +=1
  yf_ticker = yf.Ticker(stock)
  if yf_ticker.history().empty:
    continue
  


  #find splits of stock, combine into complete_stock_df
  splits_list = stock_to_split[stock]
  stock_df['Split'] = 1
  if len(splits_list) > 0:
    first_date = stock_df.loc[0].at["Action Date"]
    splits_df = pd.DataFrame(splits_list)
    splits_df.reset_index(inplace=True)
    splits_df['Action'] = 'Split'
    splits_df['Type'] = 'Stock'
    splits_df['Ticker'] = stock
    splits_df['Cost'] = 0
    splits_df['Quantity'] = 0
    splits_df['Commission'] = 0

    splits_df = splits_df.rename({'Date': 'Action Date', 'Stock Splits': 'Split'}, axis=1)  
    splits_df = splits_df[(splits_df['Action Date'] > first_date)]
    stock_df['Split'] = 1
    stock_df = pd.concat([stock_df,splits_df], axis=0, ignore_index=True)


  #find dividends of stock, combine into complete_stock_df
  dividends_list = stock_to_dividends[stock]
  if len(dividends_list) > 0:
    first_date = stock_df.loc[0].at["Action Date"]
    dividends_df = pd.DataFrame(dividends_list)
    dividends_df.reset_index(inplace=True)
    dividends_df['Action'] = 'Div'
    dividends_df['Type'] = 'Stock'
    dividends_df['Ticker'] = stock
    dividends_df['Quantity'] = 0
    dividends_df['Commission'] = 0

    dividends_df['Split'] = 1
    dividends_df = dividends_df.rename({'Date': 'Action Date', 'Dividends': 'Cost'}, axis=1)  
    dividends_df = dividends_df[(dividends_df['Action Date'] > first_date)]
    complete_stock_df = pd.concat([stock_df,dividends_df], axis=0, ignore_index=True)
  else:
    complete_stock_df = stock_df

  #clean up complete_stock_df
  complete_stock_df['Additional Quantity'] = complete_stock_df['Quantity']
  complete_stock_df = complete_stock_df.sort_values(by='Action Date')
  complete_stock_df.reset_index(inplace=True, drop = True)

  #add new columns, set up df for calcs
  complete_stock_df['Unsold Quantity'] = complete_stock_df['Quantity']

  #make the quantity negative if the action is sell
  complete_stock_df['Quantity'][complete_stock_df.Action == 'Sell'] = complete_stock_df['Quantity'][complete_stock_df.Action == 'Sell'] * -1
  complete_stock_df['Additional Quantity'][complete_stock_df.Action == 'Sell'] = 0

  complete_stock_df['Transaction Val'] = 0
  complete_stock_df.at[complete_stock_df['Action'] == 'Buy', 'Transaction Val'] = complete_stock_df.loc[complete_stock_df['Action'] == 'Buy', 'Cost'] * complete_stock_df.loc[complete_stock_df['Action'] == 'Buy', 'Quantity'] + complete_stock_df.loc[complete_stock_df['Action'] == 'Buy', 'Commission'] 

  


  #display(complete_stock_df.round(3))

  #add dother necessary columns
  complete_stock_df['Cumulative Quant'] = complete_stock_df['Quantity']
  complete_stock_df['Cumulative Buys'] = complete_stock_df['Additional Quantity'].cumsum()
  complete_stock_df['Cumulative Cost'] = complete_stock_df['Transaction Val'].cumsum()
  complete_stock_df['Gains'] = 0

  #run calculate_cumulative on each row after the first one
  complete_stock_df.apply(lambda x: calculate_cumulative(x,x.name) if x.name > 0 else x['Quantity'], axis =1)
  
  complete_stock_df['Cumulative Cost'] = complete_stock_df['Transaction Val'].cumsum()
  complete_stock_df.at[complete_stock_df['Action'] == 'Div', 'Gains'] = complete_stock_df.loc[complete_stock_df['Action'] == 'Div']['Cost'] * complete_stock_df.loc[complete_stock_df['Action'] == 'Div']['Cumulative Quant']
  complete_stock_df['Cumulative Gains'] = complete_stock_df['Gains'].cumsum()
  complete_stock_df['Cost Per Share'] = complete_stock_df['Cumulative Cost'] / complete_stock_df['Cumulative Buys'] 

  #calculate gains from each sold transaction
  complete_stock_df = complete_stock_df.fillna(0)
  display(complete_stock_df.round(3))
  
  #find the previous closing price for the stock in the market
  try:
    data = yf_ticker.history()
    previous_close_price = data['Close'].iloc[-1]  
  except:
    previous_close_price = yf_ticker.info['regularMarketPreviousClose']

  total_value_investments = previous_close_price * complete_stock_df.iloc[-1]['Cumulative Quant'] + complete_stock_df.iloc[-1]['Cumulative Gains'] 
  total_outstanding_cost = (sum(complete_stock_df[complete_stock_df['Unsold Quantity'] > 0]['Unsold Quantity'] * complete_stock_df[complete_stock_df['Unsold Quantity'] > 0]['Cost']))
  total_cost_investments = complete_stock_df.iloc[-1]['Cumulative Cost']

  rate_of_return = 100 * (total_value_investments - total_outstanding_cost) / (total_cost_investments) 
  result_lst = [[total_value_investments, total_outstanding_cost, total_cost_investments, rate_of_return]]
  stock_to_df[stock] = [complete_stock_df, pd.DataFrame(result_lst,index =[stock],columns = ['Total Value', 'Outstanding Cost', 'Total Costs','Rate of Return %'])]
  overall_df.loc[len(overall_df.index)] = [stock, total_value_investments, total_outstanding_cost, total_cost_investments, rate_of_return]



display(overall_df)

#:  1 Ticker:  AAPL


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

,Type,Action,Action Date,Ticker,Quantity,Cost,Commission,Additional Quantity,Split,Unsold Quantity,Transaction Val,Cumulative Quant,Cumulative Buys,Cumulative Cost,Gains,Cumulative Gains,Cost Per Share
0,Stock,Buy,2013-03-27,AAPL,4,65.257,7.00,4,1.0,0.0,1834.20,4,4,1834.20,0.00,0.00,458.550
1,Stock,Div,2013-05-09,AAPL,0,3.050,0.00,0,1.0,0.0,0.00,4,4,1834.20,12.20,12.20,458.550
2,Stock,Buy,2013-07-10,AAPL,3,59.943,7.00,3,1.0,0.0,1265.80,7,7,3100.00,0.00,12.20,442.857
3,Stock,Div,2013-08-08,AAPL,0,3.050,0.00,0,1.0,0.0,0.00,7,7,3100.00,21.35,33.55,442.857
4,Stock,Div,2013-11-06,AAPL,0,3.050,0.00,0,1.0,0.0,0.00,7,7,3100.00,21.35,54.90,442.857
5,Stock,Div,2014-02-06,AAPL,0,3.050,0.00,0,1.0,0.0,0.00,7,7,3100.00,21.35,76.25,442.857
6,Stock,Div,2014-05-08,AAPL,0,3.290,0.00,0,1.0,0.0,0.00,7,7,3100.00,23.03,99.28,442.857
7,Stock,Split,2014-06-09,AAPL,0,0.000,0.00,0,7.0,0.0,0.00,49,49,3100.00,0.00,99.28,63.265
8,Stock,Buy,2014-06-12,AAPL,11,93.010,7.00,11,1.0,0.0,1030.11,60,60,4130.11,0.00,99.28,68.835
9,Stock,Div,2014-08-07,AAPL,0,0.470,0.00,0,1.0,0.0,0.00,60,60,4130.11,28.20,127.48,68.835


,Stock,Total Value,Outstanding Cost,Total Costs,Rate of Return %
0,AAPL,3867.6203,0,4130.11,93.644486


In [18]:
# BLOCK 7 #


#print final results of overall portfolio analysis

overall_df = overall_df.sort_values(by=['Rate of Return %'],ascending=False, ignore_index = True).round(3)
display(overall_df)

import yfinance as yf
from datetime import date

# VOO: Vanguard 500 Index Fund ETF
# ^GSPC: S&P 500
comparison_index  = "VOO"
yf_ticker = yf.Ticker(comparison_index)

first_date = str(stock_timeline.loc[1]['Action Date'].date())
hist = yf_ticker.history(start=first_date)
last_sp_close = hist.iloc[-1]['Close']
first_sp_close = hist.iloc[0]['Close']


sp_return = 100 * (last_sp_close - first_sp_close )/ first_sp_close

portfolio_report_df = pd.DataFrame(columns=['Start Date','End Date','Total Value', 'Outstanding Cost', 'Total Costs', 'Rate of Return %',comparison_index + ' Index Rate of Return %'])
portfolio_report_df.loc[len(portfolio_report_df.index)] = [stock_timeline.loc[1]['Action Date'], date.today(),overall_df['Total Value'].sum(),overall_df['Outstanding Cost'].sum(),overall_df['Total Costs'].sum(), 100*((overall_df['Total Value'].sum() - overall_df['Outstanding Cost'].sum())/overall_df['Total Costs'].sum()), sp_return]
display(portfolio_report_df.round(3))



,Stock,Total Value,Outstanding Cost,Total Costs,Rate of Return %
0,AAPL,3867.62,0,4130.11,93.644


,Start Date,End Date,Total Value,Outstanding Cost,Total Costs,Rate of Return %,VOO Index Rate of Return %
0,2002-06-08,2022-08-31,3867.62,0,4130.11,93.644,352.55
